<a href="https://colab.research.google.com/github/Saiprapul/sentiment-analysis-imdb-transformer/blob/main/IMDB_Sentiment_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
!pip install transformers datasets
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler, DataCollatorWithPadding
from datasets import load_dataset
from tqdm.auto import tqdm

# Step 1: Load the IMDB dataset
import os
os.system('pip install datasets')
dataset = load_dataset('imdb')

# Step 2: Load the tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Step 3.1: Tokenize and prepare the dataset
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Step 3.2: Prepare DataLoader
# Use a smaller subset of the dataset for faster training
#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
#small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

# Create DataLoaders with smaller datasets and increased batch size
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator)
test_dataloader = DataLoader(tokenized_datasets["test"], batch_size=8, collate_fn=data_collator)

# Step 3.3: Fine-tune the model
# Set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3  # Reduce the number of epochs for faster training
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Set up device and start training
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Optimizer step and learning rate scheduler update
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)

print("Model fine-tuning completed!")


In [ ]:
# Step 4: Evaluation
!pip install evaluate
from evaluate import load

# Set model to evaluation mode
model.eval()

# Load metrics for evaluation
accuracy_metric = load("accuracy")
f1_metric = load("f1")

# Run the test loop
for batch in test_dataloader:
    # Move batch to device
    batch = {k: v.to(device) for k, v in batch.items()}

    # Forward pass (no gradient calculation)
    with torch.no_grad():
        outputs = model(**batch)

    # Get logits and calculate predictions
    logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)

    # Add predictions and true labels to metrics
    accuracy_metric.add_batch(predictions=predictions, references=batch["labels"])
    f1_metric.add_batch(predictions=predictions, references=batch["labels"])

# Compute final metrics
accuracy = accuracy_metric.compute()["accuracy"]
f1 = f1_metric.compute()["f1"]

print(f"Test Accuracy: {accuracy:.2f}")
print(f"Test F1-Score: {f1:.2f}")

In [12]:
# Set model to evaluation mode
model.eval()

# Custom review to test
review = "The movie was just okay. Nothing special, but nothing too bad either."

# Step 1: Tokenize the input review
inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True)

# Step 2: Move inputs to the appropriate device (CPU/GPU)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Step 3: Run the model on the tokenized review
with torch.no_grad():
    outputs = model(**inputs)

# Step 4: Get logits and calculate predictions
logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)

# Step 5: Interpret the result
sentiment = "positive" if predictions.item() == 1 else "negative"
print(f"The sentiment of the review is: {sentiment}")


The sentiment of the review is: negative
